In [34]:
import pandas as pd
import time

In [35]:
sandp = pd.read_excel("S&P_Ticker_Symbols.xlsx")
sandp.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaT,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


#### Extact and Pre-Process Company Names

In [36]:
def clean(company):
    to_remove = ["Inc.", "Inc", "Corp", "plc"]
    for item in to_remove:
        if company.endswith(item):
            company = company[:-len(item)].strip()
    return company
companies = [clean(item) for item in list(sandp['Security'])]

#### Pull Twitter with these keywords

In [46]:
import requests
import json
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream
import config #.py file with access tokens


# Variables that contains the user credentials to access Twitter API 
ACCESS_TOKEN = config.ACCESS_TOKEN 
ACCESS_SECRET = config.ACCESS_SECRET
CONSUMER_KEY = config.CONSUMER_KEY
CONSUMER_SECRET = config.CONSUMER_SECRET

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

tweet_df = pd.DataFrame()

tweet_id = []
text = []
created_at = []
place = []
user = []
hashtags_list = []
company_list = []

counter = 0 #will need to sleep for 15 minutes after each 180 requests
for company in companies:
    counter +=1
    if counter/180 ==0:
        print("Sleeping for 15 minutes because of rate limit!")
        time.sleep(1000)
    twitter = Twitter(auth=oauth)
    data=twitter.search.tweets(q=company, count=2000)
    for tweet in data["statuses"]:
        tweet_id.append(tweet['id'])
        created_at.append(tweet['created_at']) 
        text.append(tweet['text'])
        place.append(tweet['place']) 
        user.append(tweet['user']['name']) 
        company_list.append(company)
        hashtags = []
        for hashtag in tweet['entities']['hashtags']:
            hashtags.append(hashtag['text'])
        hashtags_list.append(hashtags)
tweet_df = pd.DataFrame({"tweet_id":tweet_id, "text": text, "created_at": created_at, 'place':place, 'user': user,
          'hashtags':hashtags_list, 'company': company_list})
tweet_df.to_csv("all_tweets.csv")
tweet_df.head()

,company,created_at,hashtags,place,text,tweet_id,user
0,O'Reilly Automotive,Fri Oct 13 16:16:09 +0000 2017,[],None,Potential O'Reilly Automotive (ORLY) Trade Has...,918872999373299714,kbc
1,O'Reilly Automotive,Fri Oct 13 16:00:19 +0000 2017,[],None,Potential O'Reilly Automotive (ORLY) Trade Has...,918869015430176768,EIN Auto Industry
2,O'Reilly Automotive,Fri Oct 13 15:31:01 +0000 2017,[],None,"Cubist Systematic Strategies LLC Sells 3,985 S...",918861639880933376,Dakota Financial
3,O'Reilly Automotive,Fri Oct 13 14:47:51 +0000 2017,[],None,California State Teachers Retirement System Se...,918850777711562752,Daily Political
4,O'Reilly Automotive,Fri Oct 13 14:46:18 +0000 2017,[],None,California State Teachers Retirement System Se...,918850388991840256,WKRB News
